In [1]:
# !nvidia-smi

# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
# CUDA = format_cuda_version(CUDA_version)

In [2]:
import json
import os
# import uuid  # https://docs.python.org/3/library/uuid.html
# import structlog  # for event logging
# # from dotenv import load_dotenv # enviornment vars if we want

# from pygtail import Pygtail
# import boto3
# from minio import Minio
# from dotenv import load_dotenv
from datetime import timedelta, datetime
from time import sleep
from sys import argv
# import threading
# from smart_open import smart_open
import gensim
import gensim.corpora as corpora

# setup to import the preprocessor
import sys
# from botocore.errorfactory import ClientError
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import torch_geometric
from functools import reduce
from tqdm import tqdm

from Preprocessing import Preprocessor
preprocessor = Preprocessor(0)
tqdm.pandas()


c:\Python310\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
c:\Python310\lib\site-packages\joblib\backports.py:7: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion


In [3]:
joined_exists = os.path.exists('khan_joined.csv')
khan = pd.DataFrame([])
if joined_exists:
    khan = pd.read_csv('khan_joined.csv')
khan.head(5)

""


In [4]:
if not joined_exists:
    computing = pd.read_csv("Datasets\\KhanAcademy\\Computing.csv")
    computing = computing.dropna()
    computing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263 entries, 0 to 277
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       263 non-null    object
 1   unit         263 non-null    object
 2   lesson       263 non-null    object
 3   video_title  263 non-null    object
 4   about        263 non-null    object
 5   transcript   263 non-null    object
dtypes: object(6)
memory usage: 14.4+ KB


In [5]:
if not joined_exists:
    economics = pd.read_csv("Datasets\\KhanAcademy\\Economics.csv")
    economics = economics.dropna()
    economics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 819 entries, 0 to 819
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       819 non-null    object
 1   unit         819 non-null    object
 2   lesson       819 non-null    object
 3   video_title  819 non-null    object
 4   about        819 non-null    object
 5   transcript   819 non-null    object
dtypes: object(6)
memory usage: 44.8+ KB


In [6]:
if not joined_exists:
    humanities = pd.read_csv("Datasets\\KhanAcademy\\Humanities.csv")
    humanities = humanities.dropna()
    humanities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1064 entries, 0 to 1152
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       1064 non-null   object
 1   unit         1064 non-null   object
 2   lesson       1064 non-null   object
 3   video_title  1064 non-null   object
 4   about        1064 non-null   object
 5   transcript   1064 non-null   object
dtypes: object(6)
memory usage: 58.2+ KB


In [7]:
if not joined_exists:
    math = pd.read_csv("Datasets\\KhanAcademy\\Math.csv")
    math = math.dropna()
    math.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3351 entries, 0 to 3366
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       3351 non-null   object
 1   unit         3351 non-null   object
 2   lesson       3351 non-null   object
 3   video_title  3351 non-null   object
 4   about        3351 non-null   object
 5   transcript   3351 non-null   object
dtypes: object(6)
memory usage: 183.3+ KB


In [8]:
if not joined_exists:
    science = pd.read_csv("Datasets\\KhanAcademy\\Science.csv")
    science = science.dropna()
    science.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2764 entries, 0 to 2789
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       2764 non-null   object
 1   unit         2764 non-null   object
 2   lesson       2764 non-null   object
 3   video_title  2764 non-null   object
 4   about        2764 non-null   object
 5   transcript   2764 non-null   object
dtypes: object(6)
memory usage: 151.2+ KB


In [9]:
ted_main = pd.read_csv("Datasets\\TEDTalksDataset\\ted_main.csv")
transcripts = pd.read_csv("Datasets\\TEDTalksDataset\\transcripts.csv")
validation = ted_main.join(transcripts, lsuffix='url', rsuffix='url', sort=True)
validation = validation.dropna()
validation.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2461 entries, 0 to 2466
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   comments            2461 non-null   int64 
 1   description         2461 non-null   object
 2   duration            2461 non-null   int64 
 3   event               2461 non-null   object
 4   film_date           2461 non-null   int64 
 5   languages           2461 non-null   int64 
 6   main_speaker        2461 non-null   object
 7   name                2461 non-null   object
 8   num_speaker         2461 non-null   int64 
 9   published_date      2461 non-null   int64 
 10  ratings             2461 non-null   object
 11  related_talks       2461 non-null   object
 12  speaker_occupation  2461 non-null   object
 13  tags                2461 non-null   object
 14  title               2461 non-null   object
 15  urlurl              2461 non-null   object
 16  views               2461

In [10]:
if not joined_exists:
    computing['course'].value_counts()

In [11]:
if not joined_exists:
    economics['course'].value_counts()

In [12]:
if not joined_exists:
    humanities['course'].value_counts()

In [13]:
if not joined_exists:
    math['course'].value_counts()

In [14]:
if not joined_exists:
    science['course'].value_counts()

In [15]:
if not joined_exists:
    khan_dfs = [computing, economics, humanities, math, science]
    khan = pd.concat(khan_dfs, axis=0)
    khan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8261 entries, 0 to 2789
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       8261 non-null   object
 1   unit         8261 non-null   object
 2   lesson       8261 non-null   object
 3   video_title  8261 non-null   object
 4   about        8261 non-null   object
 5   transcript   8261 non-null   object
dtypes: object(6)
memory usage: 451.8+ KB


In [16]:
if not joined_exists:
    # remap the courses to more broad categories: https://stackoverflow.com/a/16476974
    labels = ['physics', 'chemistry', 'biology', 'algebra', 'geometry', 'statistics', 'calculus', 'history', 'macroeconomics', 'microeconomics']
    for lbl in labels:
        for index, row in khan.iterrows():
            if lbl in row['course'].lower():
                row['course'] = lbl
    khan['course'].value_counts()

In [17]:
if not joined_exists:
    khan.tail(2)

# Dataset Identification
It is necessary to now identify components of the dataset that can be used for the graphical machine learning. This means identifying Nodes, Edges, Node Features, and Labels. It also includes optionally including edge weights and edge features. For the sake of simplicity I think that I will be forgoing the edge weights and edge features.

I am attempting to basically do topic modeling, but without the keywords and topics that would be customary of Latent Dirchlet Allocation. Instead the goal is to train a Graph ML model using the Khan academy data.

There are two basic routes I could take. I could perform node level prediction by treating each transcript individually and seeing which is the closest match during prediction. The other option would be to perform graph level prediction by storing all of the similarly labeled transcripts together and then using the shape of the graph for comparison. 

- Nodes (Items, People, Locations, Cars, ETC)
- Edges (Connections, Interactions, Similarity, ETC)
    - Levenshtein distance over titles?
    - Number of similar named entities from SpaCy NER?
- Node Features (Attributes)
- Labels (Node-Level, Edge-Level, Graph-Level, etc)
    - I am going to first try node-level prediction as it makes more sense to me. For this I am going to use the 'course' feature in the above pandas dataframe. This will be the target that I try to predict.

In [18]:
if not joined_exists:
    # It takes 21 min to run SpaCy preprocessing over each record in the data
    khan['transcript_cleaned'] = khan['transcript'].progress_apply(lambda x: preprocessor.clean(x))
    print(khan['transcript_cleaned'][0])

100%|██████████| 8261/8261 [19:35<00:00,  7.03it/s]  

0    [turn--, form, listen, car, drive, think, pati...
0    [strongly, gain, real, american, smith, promot...
0    [physicist, perfect, content, day, information...
0    [start, code, simple, figure, information, reg...
0    [purpose, content, call, question, think, biol...
Name: transcript_cleaned, dtype: object


In [19]:
def str_to_list(s: str) -> list:
    s = s.replace('[', '')
    s = s.replace(']', '')
    s = s.replace('\'', '')
    s = s.split(', ')
    return s

In [20]:
if not joined_exists:
    # clear up the lists to be unique only.
    khan['transcript_cleaned'] = khan['transcript'].progress_apply(lambda x: list(set(str_to_list(x))))
    print(len(khan['transcript_cleaned'][0]))
    print(khan['transcript_cleaned'][0])

100%|██████████| 8261/8261 [00:00<00:00, 21342.69it/s]

5
0    [youll be able to learn other\r\nprogramming l...
0    [or to know when the math is going into a dire...
0    [on this\r\ntiny planet buzzing with life? Why...
0    [then that means that we definitely have the t...
0    [a narrative thats been going on for millions ...
Name: transcript_cleaned, dtype: object


In [21]:
if not joined_exists:
# if not False:
    khan['transcript_n_entries'] = khan['transcript_cleaned'].progress_apply(lambda x: len(x))
    khan.head(5)

100%|██████████| 8261/8261 [00:00<00:00, 826120.48it/s]


In [22]:
# khan = khan.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])

In [23]:
if not joined_exists:
# if not False:
    khan.to_csv("khan_joined.csv", index=False)

In [24]:
# https://www.kaggle.com/code/pavansanagapati/knowledge-graph-nlp-tutorial-bert-spacy-nltk/notebook

"""
    nodes closer to yourself are more important. Take the incoming embeddings and dotproduct them with your own embedding.
    Softmax across all the dot products of the nodes coming in to have it be a probability.
    
    Need to build the corpus of all input words.
    
    Use spacy to get corpus for each doc with stopwords stripped and bigrams / trigrams joined. 
    
    Could use skipgram or bag of words with word2vec to embed words.
"""

'\n    nodes closer to yourself are more important. Take the incoming embeddings and dotproduct them with your own embedding.\n    Softmax across all the dot products of the nodes coming in to have it be a probability.\n    \n    Need to build the corpus of all input words.\n    \n    Use spacy to get corpus for each doc with stopwords stripped and bigrams / trigrams joined. \n    \n    Could use skipgram or bag of words with word2vec to embed words.\n'

In [25]:
khan.head(5)

,course,unit,lesson,video_title,about,transcript,transcript_cleaned,transcript_n_entries
0,Computer programming,Intro to JS: Drawing & Animation,Intro to programming,What is Programming?,Programming is the process of creating a set o...,"Hi, welcome to programming! If you've never le...",[youll be able to learn other\r\nprogramming l...,24
1,Computer programming,Intro to JS: Drawing & Animation,Coloring,The Power of the Docs,Created by Pamela Fox.,Voiceover: Ok so you've\r\nmade a few programs...,"[ok, perfect, and lets go back to my smiley......",90
5,Computer programming,Intro to HTML/CSS: Making webpages,Further learning,HTML validation,Learn how to validate your webpages with the W...,"- [Voiceover] On Khan Academy, we pop up the o...","[upload a file, we pop up the oh noes guide to...",13
6,Computer programming,Intro to SQL: Querying and managing data,SQL basics,Welcome to SQL,SQL is useful for creating and querying relati...,- [Instructor] The world is full of data. Ever...,[a particular\r\nimplementation of it. You won...,20
7,Computer programming,Intro to SQL: Querying and managing data,SQL basics,S-Q-L or SEQUEL?,How is it pronounced? Why? Let's discuss...,"At this point, you've probably heard me\r\npro...","[SEQUEL,\r\nwas later changed to SQL, and it s...",21


In [26]:
khan['course'].value_counts()

algebra                                         1287
calculus                                         947
chemistry                                        909
history                                          769
biology                                          757
physics                                          743
statistics                                       626
geometry                                         345
Finance and capital markets                      317
macroeconomics                                   277
microeconomics                                   225
Electrical engineering                           198
Pixar in a Box                                   175
US government and civics                         139
Cosmology and astronomy                           89
AP®︎/College US Government and Politics           87
Trigonometry                                      76
Differential equations                            70
Storytelling                                  

In [27]:
# khan['transcript_cleaned'] = khan['transcript']

In [28]:
corpus = khan['transcript_cleaned'].sum(axis=0)
if isinstance(corpus, str):
    # means it performed string concatenation so it needs to be cleaned up
    print(corpus[0:100])
    corpus = str_to_list(corpus)
    print(corpus[0: 25])
    print(len(corpus))
    print(type(corpus))

In [29]:
# convert the corpus to a set since there should be no unique values
corpus = set(corpus)
print(len(corpus))
corpus = list(corpus)
print(corpus[0:25])

380582
['', 'if I have a boulder,\r\nor a rock', 'hydroxide on the left side functioned as a base', 'they push this piston open,\r\nyou actually have work', 'but Im gonna take the\r\nconstant out of the integral. So Ill put a -7 here. And to help us solve this,\r\nand this could be a 1', 'we get our change\r\nin time is equal to the horizontal component\r\nof our displacement divided by 5 square roots of 2. But we also know that\r\nthe horizontal component of our displacement\r\nis the square root of 3 times the vertical\r\ncomponent of our displacement. Here I explicitly wrote\r\nthe magnitude notation. When we start dealing with\r\neither just the vertical or the horizontal\r\ncomponent', 'seven triangles. And the area of each of those triangles is once again ab over two. Now', 'in your 50 visits,\r\nyou dont see more', 'a corgi looks pretty\r\ndifferent from a greyhound. They were bred to be different. Corgis were bred to herd\r\nanimals and guard farmhouses', 'we are a competitive 

In [30]:
num_topics = len(khan['course'].value_counts())
print(num_topics)

26


In [31]:
%%time
# LDA Normal
temp = [d.split() for d in corpus]
print(type(temp))
words = corpora.Dictionary(temp)
words.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
corpus = [words.doc2bow(doc) for doc in temp]

<class 'list'>
CPU times: total: 13.5 s
Wall time: 13.4 s


In [32]:
%%time
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=num_topics,
                                           random_state=2,
                                           update_every=1,
                                           passes=15,
                                           alpha='auto')

CPU times: total: 11min 57s
Wall time: 11min 59s


In [33]:
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, words, mds='mmds', R=10)

c:\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [34]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13     0.507808 -0.022057       1        1  22.240762
23    -0.507127 -0.056146       2        1  12.501515
19     0.213367  0.048950       3        1  11.735393
11    -0.120710 -0.013897       4        1  10.113538
20    -0.447447 -0.238409       5        1   3.302306
16     0.102299 -0.205352       6        1   2.992253
4     -0.473579  0.123191       7        1   2.574045
0      0.319204 -0.187321       8        1   2.284133
15     0.471197 -0.203674       9        1   2.223099
21     0.483567  0.170020      10        1   2.203470
8      0.354654  0.375510      11        1   2.200106
24     0.168100 -0.459225      12        1   2.184369
10    -0.336371 -0.354118      13        1   2.036175
3     -0.211493 -0.432182      14        1   1.951187
25    -0.224642  0.242248      15        1   1.941270
5     -0.420647  0.285840      16        1   1.905020
6      0.022011  0.204608      17        1   1.864054
7     -0.059769 -0.348542      18        1   1.860214
2      0.040487  0.494364      19        1   1.730582
18    -0.109051  0.463937      20        1   1.632847
14    -0.277937  0.429384      21        1   1.616730
1     -0.312602 -0.068972      22        1   1.459403
9     -0.011663 -0.521208      23        1   1.430698
12     0.327331 -0.395028      24        1   1.398673
17     0.312946  0.218261      25        1   1.389504
22     0.190068  0.449817      26        1   1.228655, topic_info=             Term           Freq          Total Category  logprob  loglift
13            the  381297.000000  381297.000000  Default  10.0000  10.0000
41             to  240325.000000  240325.000000  Default   9.0000   9.0000
63             is  183652.000000  183652.000000  Default   8.0000   8.0000
1               a  156552.000000  156552.000000  Default   7.0000   7.0000
35             of  215149.000000  215149.000000  Default   6.0000   6.0000
...           ...            ...            ...      ...      ...      ...
91           two.    2243.844674    2244.892074  Topic26  -3.6317   4.3988
1028         blue    1873.069068    1874.116468  Topic26  -3.8123   4.3987
1694       point.    1849.239621    1850.287021  Topic26  -3.8251   4.3987
841      original    1545.211635    1546.259035  Topic26  -4.0047   4.3986
1745  temperature    1468.006353    1469.053753  Topic26  -4.0560   4.3985

[273 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
330      25  0.999882     -
1336      9  0.999650     0
21        9  0.999931     1
967      21  0.999528    12
366       9  0.999860     2
...     ...       ...   ...
344      10  0.999961     x
345      10  0.999585    x.
511      10  0.999816     y
20        4  0.999987   you
824       3  0.999882  zero

[266 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 24, 20, 12, 21, 17, 5, 1, 16, 22, 9, 25, 11, 4, 26, 6, 7, 8, 3, 19, 15, 2, 10, 13, 18, 23])

In [35]:
validation.head(5)

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,urlurl,views,transcript,urlurl
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...


In [36]:
new_text_corpus = words.doc2bow(validation['transcript'][0].split())
print(len(new_text_corpus))
prediction = lda_model.get_document_topics(new_text_corpus)
prediction.sort(key = lambda x: x[1], reverse = True)
print(prediction)
prediction = prediction[0][0]
print("Predicted Topic: %d" % prediction)
lda_model.show_topic(3)

706
[(13, 0.31274027), (23, 0.11078116), (11, 0.104480095), (20, 0.081798345), (19, 0.07991058), (16, 0.04497278), (5, 0.033084776), (8, 0.028214343), (3, 0.020512016), (4, 0.016563522), (7, 0.016006732), (10, 0.015541423), (14, 0.015479436), (25, 0.015114444), (22, 0.014572686), (18, 0.013722224), (1, 0.012820459), (17, 0.010963333), (2, 0.010611181), (6, 0.010162362)]
Predicted Topic: 13


[('as', 0.25389996),
 ('same', 0.1086497),
 ('thing', 0.076699845),
 ('A', 0.051546857),
 ('vector', 0.046303567),
 ('whole', 0.029317237),
 ('B', 0.026969587),
 ('length', 0.022040172),
 ('long', 0.01605619),
 ('isnt', 0.014599467)]